In [3]:
import os
import re
import pandas as pd
import json

In [95]:
def process_match(value):
    if(value[0]=="{" or value[-1]=="}"):
        return value
    value = value.replace(",", "")
    if(len(value)!=0 and value[0] == "(" and value[-1] == ")"):
        value = value.replace("(", "-").replace(")", "")
    return value

In [96]:
def extract_financial_data(file_path):
    
#EXTRACTING LINES OF THE FILE.

    line_array=[]
    f = open(file_path, "r")
    for line in f.readlines():
        line_array.append((" ").join(line.split()))

#FIND THE STARTING AND ENDING INDEX OF FINANCIAL DATA.

    my_regex = re.compile(r'(\d(\d)?\s+)(\w+)\s+\d\d\d\d$')
    found=False
    bounding_string=''
    for line in line_array:
        match = my_regex.search(line)
        if match and found == False:
            bounding_string=match.group()
            found=True
    
#EXTRACTING THE LINES CONTAINING FINANCIAL DATA.

    financial_data=line_array
    if(bounding_string!=''):
        bounding_index=[line_array.index(l) for l in line_array if bounding_string in l]
        if(len(bounding_index)==1):
            financial_data=financial_data[bounding_index[0]+1:]
        if(len(bounding_index)>1):
            financial_data = financial_data[bounding_index[0]+1:bounding_index[1]]
        for line in financial_data:
            if("statement" in line.lower() or "information" in line.lower() or "notes to the accounts" in line.lower()):
                financial_data = financial_data[:financial_data.index(line)]
                break
                
    if(len(financial_data)>15):
        financial_data=financial_data[:15]
    
# CODE TO FIND THE LINE - NOTES|NOTE (OPTIONAL) YEAR1 (OPTIONAL) YEAR2

    my_regex_1 = re.compile(r'((Note|Notes)\s)?((\d){4}\s)?(\d){4}$')
    str_arr=[]
    for line in financial_data:
        match = my_regex_1.search(line)
        if match:
            financial_data=financial_data[financial_data.index(line)+2:]
            str_arr=match.group().split()
            if(str_arr[0]=="Notes" or str_arr[0]=="Note"):
                str_arr.pop(0)
            break
    
#EXTRACT THE FINAL DATA AND CREATE A DICTIONARY IF POSSIBLE.

    index=-1
    financial_record={}
    if(len(str_arr)==2):
        regex_1=re.compile(r'(.*)\s(\(?\{?[\,\.0-9]+\)?\}?|-)\s(\(?\{?[\,\.0-9]+\)?\}?|-)$')
        regex_2=re.compile(r'(\(?\{?[\,0-9]+\)?\}?|-)\s(\(?\{?[\,0-9]+\)?\}?|-)$')
        regex_3=re.compile(r'(.*\s.*)$')
    elif(len(str_arr)==1):
        regex_1=re.compile(r'(.*)\s(\(?\{?[\,\.0-9]+\)?\}?|-)$')
        regex_2=re.compile(r'(\(?\{?[\,\.0-9]+\)?\}?|-)$')
        regex_3=re.compile(r'(.*\s.*)$')
    if(len(str_arr)!=0):
        if '2019' in str_arr:
            index=str_arr.index("2019")
        for line in financial_data:
            label='nan'
            value='nan'
            if(regex_1.search(line)):
                match=regex_1.search(line)
                label=match.group(1).replace("Â£","&#163").replace("Â$","&#36").replace("Â€","&#8364")
                if(index!=-1):
                    value=process_match(match.group(index+2))
            elif(regex_2.search(line)):
                match=regex_2.search(line)
                if(index!=-1):
                    value=process_match(match.group(index+1))
            elif(regex_3.search(line)):
                match=regex_3.search(line)
                label=match.group(1).replace("Â£","&#163").replace("Â$","&#36").replace("Â€","&#8364")
            financial_record.update({str(label):str(value)})
                
    return json.dumps(financial_record)

In [98]:
def extract_balance_sheets_data(dir_path):
    file_names = os.listdir(dir_path)
    result=pd.DataFrame(columns=["Filename", "Extracted Values"])
    for file in file_names:
        extracted_dic = extract_financial_data(dir_path+"\\"+file)
        file_dic={
            "Filename":str(file.split(".")[0]),
            "Extracted Values":extracted_dic
        }
        result = result.append(file_dic, ignore_index="True")
    return result

In [99]:
dir_path = ".\HCL ML Challenge Dataset"
result=extract_balance_sheets_data(dir_path)

In [100]:
result.to_csv("test_25.csv",index=False)